# Simple electricity market: Europe

## Single bidding zone with fixed load, one period

In this example we consider a single market bidding zone, Germany, France and Belgium.

The inelastic load has essentially infinite marginal utility (or higher than the marginal cost of any generator).

#### 1)Import Packages

In [1]:
import pypsa
import pandapower as pp
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import warnings
import subprocess
from shapely.errors import ShapelyDeprecationWarning
import logging

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("pypsa.pf").setLevel(logging.WARNING)
plt.rc("figure", figsize=(10, 8))

#### 2)Create a network and set Snapshots

In [2]:
# Create a new PyPSA network
network = pypsa.Network()
network.set_snapshots(range(1))  # Solve for a year 365*24

In [3]:
type(network)

pypsa.components.Network

#### 3)Add Buses

In [4]:
# Read excel file which contains Non renewable generators data
bus=pd.read_excel("data_Europe.xlsx",sheet_name='buses',header=0)

In [5]:
bus

,bus,v_nom,carrier,x,y
0,Germany,380,AC,10.4515,51.1657
1,France,380,AC,2.2137,46.2276
2,Belgium,380,AC,4.4699,50.5039


In [6]:
for index, row in bus.iterrows():
    network.add(
        "Bus",
        name=row['bus'],
        v_nom=row['v_nom'],
        carrier=row['carrier'],
        x=row['x'],
        y=row['y']
    )

In [7]:
network.buses

,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
Germany,380.0,,10.4515,51.1657,AC,,1.0,0.0,inf,PQ,,
France,380.0,,2.2137,46.2276,AC,,1.0,0.0,inf,PQ,,
Belgium,380.0,,4.4699,50.5039,AC,,1.0,0.0,inf,PQ,,


#### 4)Add generators

In [8]:
# Read excel file which contains Non renewable generators data
generators=pd.read_excel('data_Europe.xlsx',sheet_name='generators')

In [9]:
generators

,generator,p_nom,p_nom_max,bus,p_nom_extendable,marginal_cost,lifetime,efficiency,start_up_cost,shut_down_cost,min_up_time,min_down_time,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down
0,Germany Wind,6,6,Germany,False,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Germany Coal,5,20,Germany,True,30,NaN,NaN,NaN,NaN,4.0,3.0,0.54,0.75,NaN,NaN
2,Germany Gas,10,20,Germany,True,50,NaN,NaN,NaN,NaN,2.0,2.0,0.75,0.75,NaN,NaN
3,France Gas,8,15,France,True,65,NaN,NaN,NaN,NaN,2.0,2.0,0.75,0.75,NaN,NaN
4,France Wind,1,1,France,False,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,France Hydro,4,6,France,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Belgium Solar,3,3,Belgium,False,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Belgium Gas,3,5,Belgium,True,60,NaN,NaN,NaN,NaN,2.0,2.0,0.75,0.75,NaN,NaN


In [10]:
from pivottablejs import pivot_ui

ModuleNotFoundError: No module named 'pivottablejs'

In [ ]:
network.madd(
    "Generator",
    names= generators.generator,
    bus=list(generators.bus),
    carrier=list(generators.generator),
    p_nom=list(generators.p_nom),
    p_nom_max=list(generators.p_nom_max),  # MW
    p_nom_extendable=list(generators.p_nom_extendable),
    marginal_cost=list(generators.marginal_cost),
)
network.generators

In [ ]:
network.generators

#### 5)Add load

In [ ]:
# Read excel file which contains Non renewable generators data
load=pd.read_excel('data_Europe.xlsx',sheet_name='load',header=0)

In [ ]:
load

In [ ]:
for index, row in load.iterrows():
    network.add(
        "Load",
        name=row['bus'],
        bus=row['bus'],
        p_set=row['load'],
    )

In [ ]:
network.loads

#### 6)Add Transmission lines

In [ ]:
# Read excel file which contains Non renewable generators data
lines=pd.read_excel('data_Europe.xlsx',sheet_name='lines',header=0)

In [ ]:
lines.head()

In [ ]:
# Add line connection between the buses
for index, row in lines.iterrows():
    network.add(
        "Line",
        "{}_{}".format(row['country_1'],row['country_2']),
        bus0=row['country_1'],
        bus1=row['country_2'],
        s_nom=row['cap_MW'],
        x=1,
    )

In [ ]:
network.lines

In [ ]:
network

#### 7)Solve the network

In [ ]:
network.lopf(solver_name='glpk')

#### 8)Visualization of results

In [ ]:
network.buses_t.marginal_price

In [ ]:
clearing_price_redisp = network.buses_t.marginal_price.iloc[0].max()
clearing_price_redisp

In [ ]:
network.generators.p_nom_opt

In [ ]:
network.generators.p_nom_opt.plot.bar(ylabel="MW", figsize=(8, 3))

In [ ]:
network.lines_t.p0

In [ ]:
gen = network.generators.assign(g=network.generators_t.p.mean()).groupby(["bus", "carrier"]).g.sum()
gen

In [ ]:
flow = pd.Series(10, index=network.branches().index)
flow

In [ ]:
plot_graph=network.plot(
    bus_sizes=gen/10,
    bus_colors={"north_de":"midnightblue","east_de": "black", "west_de": "gray",},
    margin=1,
    line_widths=3,
    link_widths=0,
    flow=flow,
    color_geomap=True,
    projection=ccrs.EqualEarth(),
    line_colors=network.lines_t.p0.mean().abs(),
)
plt.colorbar(plot_graph[2], fraction=0.04, pad=0.004, label="Flow in MW")
plt.show()